## Walk through of a sample PCL code:

### Lesson 1 NFS configuration Example

[nfs.py](../../nfs.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
Purpose: Script to create sn SVM by using the netapp_ontap library.
         It will create a Data Interface, Network Default, Gateway.
         It will also create a DNS Domain, NFS Server and NFS Export.
Author: Vish Hulikal
Usage: nfs.py [-h] -c CLUSTER -a AGGR_NAME, -n NODE_NAME, -vs VSERVER_NAME,
                i   -v VOLUME_NAME, -ip DATA_LIF, -g GATEWAY, -d DOMAIN, -s SEVER_IP,
                    -nm NET_MASK -se NFS_SERVER, -sh EXPORT_PATH,
                    -ep EXPORT_POLICY, [-u API_USER] [-p API_PASS]
python3.11 nfs.py: -c cluster -a aggr_name, -vs/--vserver_name, -ip/--ip_address,
                   -g/--gateway_ip, -d/--domain, -s /--server_ip, -nm
                -s/--server_ip, -nm/--net_mask -se/--nfs_server, -sh/--ex_path -ep/--ex_policy
"""


#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.volume`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/volume.html)  
  - [`netapp_ontap.resources.ip_interface`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/ip_interface.html) : IP Network Interface
  - [`netapp_ontap.resources.network_route`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/network_route.html)  
  - [`netapp_ontap.resources.dns`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/dns.html) : Displays DNS information and controls the DNS subsytem. DNS domain name and DNS servers are required parameters  
  - [`netapp_ontap.resources.nfs_service`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/nfs_service.html) : NFS Service  
  - [`netapp_ontap.resources.export_policy`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/export_policy.html) : Export Policies  
  - [`netapp_ontap.resources.export_rule`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/export_rule.html) : Export Policy Rules

In [ ]:

import argparse
from getpass import getpass
import logging
from typing import Optional

from netapp_ontap import config, HostConnection, NetAppRestError
from netapp_ontap.resources import Svm, Volume, IpInterface, NetworkRoute , Dns, NfsService, ExportPolicy, ExportRule


#### Function Definitions

These functions are defined:
  
  - `create_svm`
  - `create_data_interface`  
  - `create_route`  
  - `create_dns`  
  - `create_nfs_server`  
  - `create_export_policy`  
  - `create_export_rule`  
  - `create_cifs_share`  
  - `create_volume`  

  

In [ ]:
def create_svm(vserver_name: str, aggr_name: str) -> None:
    """Create an SVM on the specified aggregate"""

    svm = Svm.from_dict({
    'name': vserver_name,
    'aggregates': [{'name': aggr_name}],
    'nfsv3': {'enabled': "true"},
    'nfsv4': {'enabled': "false"},
    'nfsv41': {'enabled': "false"}
    })

    try:
        svm.post()
        print("SVM %s created successfully" % svm.name)
    except NetAppRestError as err:
        print("Error: SVM was not created: %s" % err)
    return


In [ ]:

def create_data_interface(vserver_name: str, interface_name: str, node_name: str, ip_address: str, ip_netmask: str) -> None:
    """Creates an SVM-scoped IP Interface"""

    data = {
        'name': interface_name,
        'ip': {'address': ip_address, 'netmask': ip_netmask},
        'enabled': True,
        'scope': 'svm',
        'svm': {'name': vserver_name},
        'port': {'name': 'e0d', 'node': node_name},
        'location': {
           'auto_revert': True,
           'broadcast_domain': {'name': 'Default'},
        }
    }

    ip_interface = IpInterface(**data)

    try:
        ip_interface.post()
        print("Ip Interface %s created successfully" % ip_interface.ip.address)
    except NetAppRestError as err:
        print("Error: IP Interface was not created: %s" % err)
    return


In [ ]:

def create_route(vserver_name: str, net_gateway_ip: str) -> None:
    """Creates a network route"""
    """The default destination will be set to "0.0.0.0/0" for IPv4 gateway addresses"""

    data = {
        'gateway': net_gateway_ip,
        'svm': {'name': vserver_name}
    }

    route = NetworkRoute(**data)

    try:
        route.post()
        print("Route %s created successfully" % route.gateway)
    except NetAppRestError as err:
        print("Error: Route was not created: %s" % err)
    return


In [ ]:

def create_dns(vserver_name: str, domain: str, dns_server_ip: str) -> None:
    """Creates a DNS server"""

    data = {
        'domains': [domain],
        'servers': [dns_server_ip],
        'svm': {'name': vserver_name}
    }

    dns = Dns(**data)

    try:
        dns.post()
        print("DNS %s created successfully" % dns.domains)
    except NetAppRestError as err:
        print("Error: DNS was not created: %s" % err)
    return


In [ ]:

def create_nfs_server(vserver_name: str, domain_name: str, nfs_server: str, server_ip: str) -> None:
    """Creates a NFS server"""

    SVM = Svm.find(name=vserver_name)

    data = {
        'name': nfs_server,
        'scope': 'svm',
        'svm': {'name': vserver_name, 'uuid': SVM.uuid},
        'protocol': {'v4_id_domain': domain_name},
        #'protocol': {'v3_enabled': bool('true')},
        'vstorage_enabled': 'true'
    }

    nfs_service = NfsService(**data)

    try:
        nfs_service.post()
        print("NFS Server %s created successfully" % nfs_server)
    except NetAppRestError as err:
        print("Error: NFS Server was not created: %s" % err)
    return


In [ ]:

def create_export_policy(vserver_name: str, ex_path: str, host_name: str, ex_policy: str) -> None:
    """Creates an export policy for an SVM"""

    SVM = Svm.find(name=vserver_name)

    data = {
        'name': ex_policy,
        'svm': {'name': vserver_name, 'uuid': SVM.uuid},
        'policy': ex_policy,
        'rules': [
            {
                'clients': [{'match': '0.0.0.0/0'}],'ro_rule': ['any'], 'rw_rule': ['any'], 'anonymous_user': 'any', 'superuser': ['any'], 'protocols': ['any']
            }
        ]
    }

    export_policy = ExportPolicy(**data)

    try:
        export_policy.patch()
        print("Export Policy for NFS Server %s created successfully" % export_policy.name)
    except NetAppRestError as err:
        print("Error: Export Policy was not created: %s" % err)
    return


In [ ]:

def create_export_rule(ex_policy: str)  -> None:
    """Creates an export rule for an SVM"""

    EP = ExportPolicy.find(name=ex_policy)

    data = {
        'policy.id': EP.id,
        'clients': [{'match': '0.0.0.0/0'}],
        'ro_rule': ['any'],
        'rw_rule': ['any'],
        'anonymous_user': 'any',
        'superuser': ['any'],
        'protocols': ['any']
    }

    export_rule = ExportRule(**data)

    try:
        export_rule.post()
        print("Export Rule for NFS Server %s created successfully" % export_rule.policy.id)
    except NetAppRestError as err:
        print("Error: Export Rule was not created: %s" % err)
    return


In [ ]:

def create_volume(volume_name: str, vserver_name: str, aggr_name: str, net_path: str, ex_policy: str, volume_size: int) -> None:
    """Creates a new volume in a SVM"""

    data = {
        'name': volume_name,
        'svm': {'name': vserver_name},
        'aggregates': [{'name': aggr_name }],
        'size': volume_size,
        'nas': {'security_style': 'unix', 'path': net_path, 'unix_permissions': 4777, 'export_policy.name': ex_policy, 'junction_parent.name': vserver_name + "_root"},   #volume_name},
        'space_guarantee': 'volume'
    }

    volume = Volume(**data)

    try:
        volume.post()
        print("Volume %s created successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume was not created: %s" % err)
    return


#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a new NFS Share for a given VServer"
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="Cluster Name"
    )
    parser.add_argument(
        "-n", "--node_name", required=True, help="Node Name"
    )
    parser.add_argument(
        "-a", "--aggr_name", required=True, help="Aggregate name"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="VServer name to create NFS Share"
    )
    parser.add_argument(
        "-v", "--volume_name", required=True, help="Volume name"
    )
    parser.add_argument(
        "-ip", "--ip_address", required=True, help="Data Interface IP Address"
    )
    parser.add_argument(
        "-g", "--gateway_ip", required=True, help="Default Gateway IP Address"
    )
    parser.add_argument(
        "-d", "--domain", required=True, help="DNS DOmain Name"
    )
    parser.add_argument(
        "-s", "--server_ip", required=True, help="DNS Server IP Address"
    )
    parser.add_argument(
        "-nm", "--ip_netmask", required=True, help="DNS Server IP Address"
    )
    parser.add_argument(
        "-se", "--nfs_server", required=True, help="NFS Server"
    )
    parser.add_argument(
        "-sh", "--ex_path", required=True, help="Export Path"
    )
    parser.add_argument(
        "-ep", "--ex_policy", required=True, help="Export Policy"
    )

    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )

    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    create_svm(args.vserver_name, args.aggr_name)
    create_data_interface(args.vserver_name, args.vserver_name+"_lif1", args.node_name, args.ip_address, args.ip_netmask)
    create_route(args.vserver_name, args.gateway_ip)
    create_dns(args.vserver_name, args.cluster, args.server_ip)
    create_nfs_server(args.vserver_name, args.domain, args.nfs_server, args.server_ip)
#    create_export_policy(args.vserver_name, args.ex_path, args.nfs_server, args.ex_policy)
#    create_export_rule(args.ex_policy)
    create_volume(args.volume_name, args.vserver_name, args.aggr_name, args.ex_path, args.ex_policy, 200000000)


---

### Verify that the svm was created:

In [ ]:
!ssh admin@cluster1 vserver show -vserver nas_svm_03

### Verify that the volume was created:

In [ ]:
!ssh admin@cluster1 volume show -vserver nas_svm_02 -volume nas_svm_03_nfs_03

### Verify that the data lif was created:

In [ ]:
!ssh admin@cluster1 net int show -vserver nas_svm_03

### Verify that the DNS Configuration was created:

In [ ]:
!ssh admin@cluster1 vserver services name-service dns show -vserver nas_svm_03

---

[NFS export policy configuration through ONTAP CLI Demo - Notebook:](./Module4_L2.ipynb)

[NFS Mount Demo - Notebook:](./Module4_L3.ipynb)

[Back to Module 4 Demo - Notebook:](./Module4_demo.ipynb)